In [63]:
## Load Packages ## 

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.optimize import minimize_scalar
from quantecon.markov import tauchen

In [64]:
## Set Parameters ## 

# Aiyagari (1994) 
pAlpha  = 0.36 
pBeta   = 0.96
pDelta  = 0.08 
pA      = 1 

# income process (log(z) = pRho*log(z(-1)) + eps)
pMu     = 0.0
pSigma  = 0.0872
pRho    = 0.90
pNz     = 7

# wealth grid   
pNa1    = 50
pNa2    = 100 # finer grid for interpolation 
minGrida = 0
maxGrida = 150

In [65]:
## Helper Functions ## 

# Invariant Distribution of MC (first-order)
def fnMCss(mT, eigvlmethod=True):
    if eigvlmethod:
        _, eigvc = np.linalg.eig(mT)
        vec = eigvc[:, 0]
        vec = vec / np.sum(vec)
        return vec 
    else:
        err = 10 
        x0 = np.zeros(len(mT))
        x0[0] = 1
        while err >= 1e-12: 
            x1 = np.dot(mT, np.transpose(x0))
            err = np.max(np.abs(x1 - x0))
            x0 = np.copy(x1)
        return x0
    
# HH Obj Function
def fnHHobj(x, beta, budget, grida, Na, expvf):
    aprime = x.item() if np.ndim(x) > 0 else x 
    c = np.maximum(1e-8, budget - aprime)

    aLow = np.searchsorted(grida, aprime, side='right') - 1 
    aLow = np.maximum(0, np.minimum(Na - 2, aLow))
    aHigh = aLow + 1

    wtLow = (grida[aHigh] - aprime) / (grida[aHigh] - grida[aLow])
    wtLow = np.clip(wtLow, 0, 1)
    wtHigh = 1 - wtLow  

    expval = wtLow*expvf[aLow] + wtHigh*expvf[aHigh]

    val = -(np.log(c) + beta * expval)
    return val

# Solver (Minimizer)
def fnOptaprime(objargs, amin, amax):
    result = minimize_scalar(fnHHobj, args=objargs, 
                             bounds=[amin, amax], method='bounded', 
                             options={'xatol': 1e-12, 'maxiter': 2000})
    return result.x

# Linear (1D) Interpolation (Grid)
def fnInterp1dGrid(x, grid, Ngrid, weights=False):
    low = np.searchsorted(grid, x, side='right') - 1
    low = np.maximum(0, np.minimum(Ngrid - 2, low))
    high = low + 1
    if weights:
        wtLow = (grid[high] - x) / (grid[high] - grid[low])
        wtLow = np.clip(wtLow, 0, 1)
        wtHigh = 1 - wtLow
        return low, high, wtLow, wtHigh
    else:
        return low, high

In [66]:
## Grids ## 

# coarsed wealth (Maliar, Maliar, and Valli, 2010)
x1 = np.linspace(0, 0.5, pNa1)
x2 = np.linspace(0, 0.5, pNa2)
y1 = x1**7 / np.max(x1**7)
y2 = x2**7 / np.max(x2**7)
vGrida1 = minGrida + (maxGrida - minGrida) * y1 
vGrida2 = minGrida + (maxGrida - minGrida) * y2

# productivity/income 
mc = tauchen(rho=pRho, sigma=pSigma**2, mu=pMu, n=pNz, n_std=3)
vZ = np.exp(mc.state_values)
mTz = mc.P
vPiz = fnMCss(np.transpose(mTz), eigvlmethod=True)

In [67]:
## Initialise GE Loop ## 

# initial guess/placeholder variables 
aggK = 6 
mVF = np.transpose(np.tile(0.01 * vGrida1, (pNz, 1)))
mVFnew = np.zeros(mVF.shape)
mPolaprime1 = np.zeros((pNa1, pNz))
mPolc = np.zeros((pNa1, pNz))
currentDist = np.ones((pNa2, pNz))
currentDist = currentDist / (pNa2 * pNz) # normalize 

# parameters for loop
tolGE = 1e-8 
tolVFI = 1e-8
tolDist = 1e-8
maxGEiter = 2000
maxVFIiter = 2000 
convergedGe = False 
convergedVFI = False
wtOldK = 0.9900

In [ ]:
## Main GE Loop - RE-SRCE Solver ##

#==================================================# 
# GE OUTER LOOP  
#==================================================#

aggL = np.dot(vPiz, vZ) # aggregate labour supply - exogenous (inelastic indiv. LS)
errGE = 20
iterGE = 1
while errGE >= tolGE:

    r = pAlpha * pA * (aggK / aggL)**(pAlpha - 1) - pDelta
    w = (1-pAlpha) * pA * (aggK / aggL)**(pAlpha)

    #==================================================#
    # VFI LOOP - INNER LOOP W/ Howard Improvement
    #==================================================#

    errVFI = 20
    iterVFI = 1 
    while errVFI >= tolVFI and iterVFI <= maxVFIiter:
        
        # expected VF 
        expVF = np.dot(mVF, np.transpose(mTz))

        # loop over income states 
        for iz in range(pNz):   
            
            expVFz = expVF[:, iz] # expected VF conditioned on current income state
            minwealth = minGrida 

            # loop over asset states 
            for ia in range(pNa1):
                
                #---------- VFI W/ Policy Function (Non-Accelerated) ----------#
                if iterVFI <= 30 or iterVFI % 20 == 0:
                    
                    # solve for optimal aprime (and c) - off the grid approach
                    budget = w*vZ[iz] + (1+r)*vGrida1[ia]
                    aprime = fnOptaprime(objargs=(pBeta, budget, vGrida1, pNa1, expVFz),
                                         amin=minwealth, amax=budget)
                    c = np.maximum(1e-8, budget - aprime)

                    # interpolate VF on the asset-grid corresponding to aprime 
                    aLow, aHigh, wtLow, wtHigh = fnInterp1dGrid(aprime, vGrida1, pNa1, weights=True)

                    value = wtLow*expVFz[aLow] + wtHigh*expVFz[aHigh]

                    # update VF and policy functions
                    mVFnew[ia, iz] = np.log(c) + pBeta * value 
                    mPolaprime1[ia, iz] = aprime 
                    mPolc[ia, iz] = c
                    
                    # update minimum wealth to take advtg. of monotonicity in policy rule 
                    minwealth = aprime 

                #---------- VFI W/O Policy Function (Accelerated) ----------#
                else: 
                    aprime = mPolaprime1[ia, iz]
                    c = mPolc[ia, iz]

                    # interpolate VF on the asset-grid corresponding to aprime 
                    aLow, aHigh, wtLow, wtHigh = fnInterp1dGrid(aprime, vGrida1, pNa1, weights=True)
               
                    value = wtLow*expVFz[aLow] + wtHigh*expVFz[aHigh]

                    # update VF 
                    mVFnew[ia, iz] = np.log(c) + pBeta * value 
        
        # compute error
        errVFI = np.max(np.abs(mVFnew - mVF))

        # update VF 
        mVF = np.copy(mVFnew)
        iterVFI += 1
    
    #==================================================#
    # INTERPOLATE POLICY RULE ON FINER GRID
    #==================================================#
     
    fnLinInterp = interp1d(vGrida1, mPolaprime1, kind='linear', fill_value='extrapolate', axis=0)
    mPolaprime2 = fnLinInterp(vGrida2)
    #mPolaprime2 = np.maximum(mPolaprime2, minGrida)

    #==================================================#
    # COMPUTE STATIONARY DISTRIBUTION
    #==================================================#
    print(f"Computing Stationary Distribution...")
    #---------- Non-stochastic Iterative (Histogram) Method ----------#
    errDist = 20
    iterDist = 1
    while errDist >= tolDist:
        newDist = np.zeros(currentDist.shape) # reset new distribution 

        for iz in range(pNz):
            for ia in range(pNa2):
                aprime = mPolaprime2[ia, iz]

                # interpolate aprime on the finer asset-grid 
                LB, UB, wtLB, wtUB = fnInterp1dGrid(aprime, vGrida2, pNa2, weights=True)

                # compute current mass over state (a,z) 
                mass = currentDist[ia, iz]

                # compute new distribution - incrementally allocate mass over LB and UB on asset-grid
                for iznext in range(pNz):
                    newDist[LB, iznext] += wtLB * mass * mTz[iz, iznext]
                    newDist[UB, iznext] += wtUB * mass * mTz[iz, iznext]
                
        
        # compute error
        errDist = np.max(np.abs(newDist - currentDist))

        # update distribution 
        currentDist = np.copy(newDist)
        iterDist += 1

    #==================================================#
    # UPDATE AGGREGATE(S) AND PRICE(S)
    #==================================================#

    # marginal distribution of wealth (a) over income states (z) 
    marginalDista = np.sum(currentDist, axis=1)

    # Capital Market Clearing - New/Endog. Capital
    aggKnew = np.dot(vGrida2, marginalDista)

    # compute error 
    errGE = np.abs(aggKnew - aggK)

    # update agg. capital 
    aggK = wtOldK*aggK + (1-wtOldK)*aggKnew
    iterGE += 1

    #==================================================#
    # PRINT PROGRESS 
    #==================================================#
    print("\n")
    print(f"GE Iteration: {iterGE}. Error: {np.round(errGE, 8)}")
    print(f"AggK: {np.round(aggK, 4)}")
    print(f"r: {np.round(r, 4)}, w: {np.round(w, 4)}")
    print("-"*50)

Computing Stationary Distribution...


GE Iteration: 2. Error: 5.999999
AggK: 5.94
r: 0.0344, w: 1.2198
--------------------------------------------------
Computing Stationary Distribution...


GE Iteration: 3. Error: 5.93999926
AggK: 5.8806
r: 0.0351, w: 1.2154
--------------------------------------------------
Computing Stationary Distribution...


GE Iteration: 4. Error: 5.8805954
AggK: 5.8218
r: 0.0359, w: 1.211
--------------------------------------------------
Computing Stationary Distribution...


GE Iteration: 5. Error: 5.82176099
AggK: 5.7636
r: 0.0366, w: 1.2066
--------------------------------------------------
Computing Stationary Distribution...


GE Iteration: 6. Error: 5.76346673
AggK: 5.7059
r: 0.0374, w: 1.2023
--------------------------------------------------
Computing Stationary Distribution...


GE Iteration: 7. Error: 5.70554841
AggK: 5.6489
r: 0.0381, w: 1.1979
--------------------------------------------------
Computing Stationary Distribution...


GE Iteration: